In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
trainA = []
trainB = []
for i in range(1,701):
  img = cv2.imread('rain/{}clean.jpg'.format(i))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(256,256))
  trainA.append(img)
  img = cv2.imread('rain/{}bad.jpg'.format(i))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(256,256))
  trainB.append(img)
trainA = np.array(trainA)
trainB = np.array(trainB)
trainA = (trainA - 127.5)/127.5
trainB = (trainB - 127.5)/127.5

In [4]:
from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, add, Concatenate
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.utils.vis_utils import plot_model
from keras.initializers import RandomNormal
from tensorflow.keras.losses import BinaryCrossentropy

def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
  # C32
	d = Conv2D(32, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
 	# C32
	d = Conv2D(32, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(d)
	d = LeakyReLU(alpha=0.2)(d)
  # C64
	d = Conv2D(64, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(d)
	d = LeakyReLU(alpha=0.2)(d)
	d = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid', data_format=None)(d)
 	# C128
	d = Conv2D(128, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	d = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid', data_format=None)(d)
 	# C256
	d = Conv2D(256, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	d = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid', data_format=None)(d)
 	# C512
	d = Conv2D(512, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
 	# 512
	d = Conv2D(512, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = Adam(lr=0.0003, beta_1=0.5)
	model.compile(loss= BinaryCrossentropy(from_logits=True), optimizer=opt, loss_weights=[0.5])
	return model 
# define image shape
image_shape = (256,256,3)
# create the model
model = define_discriminator(image_shape)
# summarize the model
model.summary()
# plot the model
#plot_model(model, to_file='/content/drive/My Drive/test/discriminator_model_plot.png', show_shapes=True, show_layer_names=True)

Using TensorFlow backend.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 256, 256, 6)  0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Con

In [5]:
# example of defining a u-net encoder-decoder generator model
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import UpSampling2D
from keras.utils.vis_utils import plot_model
from keras.backend import resize_images
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add downsampling layer
	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# conditionally add batch normalization
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g

# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add upsampling layer
	g = UpSampling2D(size=(2,2), data_format=None, interpolation='nearest')(layer_in)
	g = Conv2D(n_filters, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(g)
	# add batch normalization
	g = BatchNormalization()(g, training=True)
	# conditionally add dropout
	if dropout:
		g = Dropout(0.5)(g, training=True)
	# merge with skip connection
	g = Concatenate()([g, skip_in])
	# relu activation
	g = Activation('relu')(g)
	return g

# define the standalone generator model
def define_generator(image_shape=(256,256,3)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# encoder model: C64-C128-C256-C512-C512-C512-C512-C512
	e1 = define_encoder_block(in_image, 64, batchnorm=False)
	e2 = define_encoder_block(e1, 128)
	e3 = define_encoder_block(e2, 128)
	e4 = define_encoder_block(e3, 256)
	e5 = define_encoder_block(e4, 512)
	# bottleneck, no batch norm and relu
	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e5)
	b = Activation('relu')(b)
	# decoder model: C512-C512-C512-C512-C512-C256-C128-C64
	d1 = decoder_block(b, e5, 512)
	d2 = decoder_block(d1, e4, 256, dropout=False)
	d3 = decoder_block(d2, e3, 128, dropout=False)
	d4 = decoder_block(d3, e2, 128, dropout=False)
	d5 = decoder_block(d4, e1, 64, dropout=False)
	# output
	g = UpSampling2D(size=(2,2), data_format=None, interpolation='nearest')(d5)
	g = Conv2D(3, (4,4), strides=(1,1), padding='same', kernel_initializer=init)(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

# define image shape
image_shape = (256,256,3)
# create the model
model = define_generator(image_shape)
# summarize the model
model.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 128, 128, 64) 3136        input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_13 (LeakyReLU)      (None, 128, 128, 64) 0           conv2d_14[0][0]                  
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 64, 64, 128)  131200      leaky_re_lu_13[0][0]       

In [6]:
from tensorflow.keras.losses import BinaryCrossentropy
import keras.backend as K
from keras.applications.vgg19 import VGG19
image_shape = (256, 256, 3)

def perceptual_loss(y_true, y_pred):
    vgg = VGG19(include_top=False, weights='imagenet', input_shape=image_shape)
    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))

def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true*y_pred)

def define_gan(g_model, d_model, image_shape):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    in_target = Input(shape = image_shape)
    # connect the source image to the generator input
    gen_out = g_model(in_src)
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    # src image as input, generated image and classification output
    model = Model([in_src,in_target], [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=[wasserstein_loss, perceptual_loss], optimizer=opt, loss_weights=[100,1])
    return model

In [7]:
def generate_real_samples(n_samples, patch_shape):
	# unpack dataset

	# choose random instances
	ix = np.random.randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = np.ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

In [8]:
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = np.zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

In [9]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, d_model, gan_model, n_samples=1):
	# select a sample of input images
	[X_realA, X_realB], _ = generate_real_samples(n_samples, 1)
	# generate a batch of fake samples
	X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
	# scale all pixels from [-1,1] to [0,1]
	X_realA = (X_realA + 1) / 2.0
	X_realB = (X_realB + 1) / 2.0
	X_fakeB = (X_fakeB + 1) / 2.0
	X_fakeB = 255 * X_fakeB	
	# plot generated target image
	for i in range(n_samples):
		plt.subplot(3, n_samples, 1 + n_samples + i)
		plt.axis('off')
		plt.imshow(X_fakeB[i])
	# save plot to file
	filename1 = 'checkerboard_lr/plot_%06d.png' % (step+1)
	cv2.imwrite(filename1,X_fakeB[0])
	# save the generator, discriminator and gan models
	filename2 = 'checkerboard_lr/g_model_%06d.h5' % (step+1)
	g_model.save(filename2)
	#filename3 = 'RainLtrain/trainingresult/d_model_%06d.h5' % (step+1)
	#d_model.save(filename3)
	#filename4 = 'RainLtrain/trainingresult/gan_model_%06d.h5' % (step+1)
	#gan_model.save(filename4)
	print('>Saved: %s,and %s' % (filename1, filename2))

In [10]:
def train(d_model, g_model, gan_model, n_epochs=200, n_batch=1, n_patch=32):
	# unpack dataset
  
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		[X_realA, X_realB], y_real = generate_real_samples( n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realB, n_patch)
		# update discriminator for real samples
		d_loss1 = d_model.train_on_batch([X_realB, X_realA], y_real)
		# update discriminator for generated samples
		d_loss2 = d_model.train_on_batch([X_realB, X_fakeB], y_fake)
		# update the generator
		g_loss, _, _ = gan_model.train_on_batch([X_realB,X_realA], [y_real, X_realA])
		# summarize performance
		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
    # summarize model performance
		if (i+1) % (bat_per_epo * 1) == 0:
			summarize_performance(i, g_model,d_model, gan_model) 

In [ ]:
image_shape = (256,256,3)
# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)
# train model
train(d_model, g_model, gan_model)

/home/pytorch/anaconda3/envs/sahil/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, d1[0.262] d2[0.551] g[136.975]
>2, d1[0.230] d2[0.490] g[118.127]
>3, d1[0.267] d2[0.503] g[113.270]
>4, d1[0.262] d2[0.507] g[85.903]
>5, d1[0.261] d2[0.501] g[72.276]
>6, d1[0.289] d2[0.472] g[66.668]
>7, d1[0.277] d2[0.465] g[70.060]
>8, d1[0.281] d2[0.474] g[83.029]
>9, d1[0.283] d2[0.458] g[81.596]
>10, d1[0.288] d2[0.434] g[53.360]
>11, d1[0.295] d2[0.451] g[71.203]
>12, d1[0.292] d2[0.427] g[52.809]
>13, d1[0.291] d2[0.449] g[59.135]
>14, d1[0.287] d2[0.431] g[59.949]
>15, d1[0.282] d2[0.441] g[55.504]
>16, d1[0.279] d2[0.413] g[37.370]
>17, d1[0.304] d2[0.396] g[38.561]
>18, d1[0.305] d2[0.402] g[63.568]
>19, d1[0.320] d2[0.411] g[42.801]
>20, d1[0.312] d2[0.391] g[50.110]
>21, d1[0.313] d2[0.403] g[58.817]
>22, d1[0.318] d2[0.390] g[41.649]
>23, d1[0.312] d2[0.380] g[43.531]
>24, d1[0.318] d2[0.384] g[41.179]
>25, d1[0.313] d2[0.366] g[38.498]
>26, d1[0.320] d2[0.373] g[42.018]
>27, d1[0.321] d2[0.374] g[34.124]
>28, d1[0.315] d2[0.373] g[34.638]
>29, d1[0.316] d2[0.376] g

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


>700, d1[0.157] d2[0.347] g[9.991]
>Saved: checkerboard_lr/plot_000700.png,and checkerboard_lr/g_model_000700.h5
>701, d1[0.157] d2[0.347] g[14.303]
>702, d1[0.157] d2[0.347] g[26.153]
>703, d1[0.157] d2[0.347] g[17.862]
>704, d1[0.157] d2[0.347] g[13.293]
>705, d1[0.157] d2[0.347] g[15.755]
>706, d1[0.157] d2[0.347] g[19.712]
>707, d1[0.157] d2[0.347] g[14.736]
>708, d1[0.157] d2[0.347] g[22.841]
>709, d1[0.157] d2[0.347] g[13.301]
>710, d1[0.179] d2[0.413] g[13.224]
>711, d1[0.163] d2[0.351] g[21.365]
>712, d1[0.179] d2[0.355] g[12.152]
>713, d1[0.171] d2[0.347] g[15.267]
>714, d1[0.165] d2[0.347] g[11.203]
>715, d1[0.160] d2[0.347] g[12.071]
>716, d1[0.159] d2[0.347] g[19.829]
>717, d1[0.157] d2[0.347] g[13.438]
>718, d1[0.157] d2[0.347] g[28.770]
>719, d1[0.157] d2[0.347] g[7.564]
>720, d1[0.157] d2[0.347] g[9.283]
>721, d1[0.157] d2[0.347] g[26.719]
>722, d1[0.157] d2[0.347] g[22.033]
>723, d1[0.157] d2[0.347] g[9.950]
>724, d1[0.157] d2[0.347] g[16.325]
>725, d1[0.157] d2[0.347] 

/home/pytorch/anaconda3/envs/sahil/lib/python3.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

>18900, d1[0.157] d2[0.347] g[5.988]
>Saved: checkerboard_lr/plot_018900.png,and checkerboard_lr/g_model_018900.h5
>18901, d1[0.157] d2[0.347] g[4.911]
>18902, d1[0.157] d2[0.347] g[7.718]
>18903, d1[0.157] d2[0.347] g[7.881]
>18904, d1[0.157] d2[0.347] g[5.628]
>18905, d1[0.157] d2[0.347] g[7.238]
>18906, d1[0.157] d2[0.347] g[11.732]
>18907, d1[0.157] d2[0.347] g[6.342]
>18908, d1[0.157] d2[0.347] g[5.838]
>18909, d1[0.157] d2[0.347] g[5.660]
>18910, d1[0.157] d2[0.347] g[6.151]
>18911, d1[0.157] d2[0.347] g[8.378]
>18912, d1[0.157] d2[0.347] g[16.734]
>18913, d1[0.157] d2[0.347] g[14.310]
>18914, d1[0.157] d2[0.347] g[16.616]
>18915, d1[0.157] d2[0.347] g[11.590]
>18916, d1[0.157] d2[0.347] g[9.852]
>18917, d1[0.157] d2[0.347] g[6.287]
>18918, d1[0.157] d2[0.348] g[6.272]
>18919, d1[0.157] d2[0.347] g[12.253]
>18920, d1[0.157] d2[0.347] g[5.708]
>18921, d1[0.157] d2[0.347] g[10.051]
>18922, d1[0.157] d2[0.347] g[5.494]
>18923, d1[0.157] d2[0.347] g[8.482]
>18924, d1[0.157] d2[0.347]

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


>19600, d1[0.157] d2[0.347] g[13.547]
>Saved: checkerboard_lr/plot_019600.png,and checkerboard_lr/g_model_019600.h5
>19601, d1[0.157] d2[0.347] g[7.122]
>19602, d1[0.157] d2[0.347] g[20.597]
>19603, d1[0.157] d2[0.347] g[4.896]
>19604, d1[0.157] d2[0.347] g[7.431]
>19605, d1[0.157] d2[0.347] g[9.717]
>19606, d1[0.157] d2[0.347] g[5.824]
>19607, d1[0.157] d2[0.347] g[12.913]
>19608, d1[0.157] d2[0.347] g[12.501]
>19609, d1[0.157] d2[0.347] g[17.214]
>19610, d1[0.157] d2[0.347] g[4.756]
>19611, d1[0.157] d2[0.347] g[5.994]
>19612, d1[0.157] d2[0.347] g[8.132]
>19613, d1[0.157] d2[0.347] g[10.859]
>19614, d1[0.157] d2[0.347] g[11.582]
>19615, d1[0.157] d2[0.347] g[4.851]
>19616, d1[0.157] d2[0.347] g[4.818]
>19617, d1[0.157] d2[0.347] g[6.848]
>19618, d1[0.157] d2[0.347] g[7.223]
>19619, d1[0.157] d2[0.347] g[9.775]
>19620, d1[0.157] d2[0.347] g[5.220]
>19621, d1[0.157] d2[0.347] g[5.303]
>19622, d1[0.157] d2[0.347] g[6.688]
>19623, d1[0.157] d2[0.347] g[5.275]
>19624, d1[0.157] d2[0.347]

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


>20300, d1[0.157] d2[0.347] g[6.809]
>Saved: checkerboard_lr/plot_020300.png,and checkerboard_lr/g_model_020300.h5
>20301, d1[0.157] d2[0.347] g[13.161]
>20302, d1[0.157] d2[0.347] g[4.732]
>20303, d1[0.157] d2[0.347] g[4.620]
>20304, d1[0.157] d2[0.347] g[6.533]
